# API wrappers

The OpenWeatherMap offers REST endpoints for querying current weather, forecasts, historical data, etc. However, accessing this data directly via the REST API requires handling multiple API calls, query parameters, and response parsing. The pyowm library abstracts these complexities and provides useful built-in functionalities.

After signing in to OpenWeatherMap retrieve your api key at https://home.openweathermap.org/api_keys

You will also need to install the pyowm package: pip install pyowm 

In [2]:
!pip install pyowm

     ---------------------------------------- 4.5/4.5 MB 18.1 MB/s eta 0:00:00


In [79]:
import requests
import pyowm
import json

api_key = '1fe2d18b8130ad19d8ef94ec8da37910'

## use case 1: managing API keys

In a raw rest API call you always have to manage credentials in each individual call. Wrappers usually store and manage the authentication for you

In [80]:
#You can get current weather data by making a GET request to an endpoint like:

params = {
    'appid' : api_key
}

response = requests.get('https://api.openweathermap.org/data/2.5/weather?q=London', params = params)

json.loads(response.text)

#but for every call you make using GET from now on you do need to add the parameters, since the raw API does not manage authentication for you

{'coord': {'lon': -0.1257, 'lat': 51.5085},
 'weather': [{'id': 501,
   'main': 'Rain',
   'description': 'moderate rain',
   'icon': '10d'}],
 'base': 'stations',
 'main': {'temp': 291.41,
  'feels_like': 291.67,
  'temp_min': 290.71,
  'temp_max': 292.13,
  'pressure': 995,
  'humidity': 91,
  'sea_level': 995,
  'grnd_level': 991},
 'visibility': 10000,
 'wind': {'speed': 6.17, 'deg': 200},
 'rain': {'1h': 1.31},
 'clouds': {'all': 75},
 'dt': 1756888553,
 'sys': {'type': 2,
  'id': 2075535,
  'country': 'GB',
  'sunrise': 1756876593,
  'sunset': 1756925015},
 'timezone': 3600,
 'id': 2643743,
 'name': 'London',
 'cod': 200}

Most wrappers (pyowm included) include some way of initializing a session with the authentication key that you then don't need to type again.

Initialize pyowm with the default configuration. Thenopen the weather manager

Check out a snippet here: https://pyowm.readthedocs.io/en/latest/v3/code-recipes.html#weather_data

In [81]:
from pyowm.owm import OWM
owm = OWM(api_key)
weather_mgr = owm.weather_manager()
weather_mgr

<pyowm.weatherapi30.weather_manager.WeatherManager>

## use case 2: Simplified calls

With the raw REST API, you'd have to build a URL manually, send the request, and parse the JSON response to get the current weather.

In [82]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'

response = requests.get(url,params= params)
data = response.json()
temperature = data['main']['temp']
humidity = data['main']['humidity']
wind_speed = data['wind']['speed']

print(f"Temperature: {temperature}°C, Humidity: {humidity}%, Wind Speed: {wind_speed} m/s")

Temperature: 291.41°C, Humidity: 91%, Wind Speed: 6.17 m/s


Get the equivalent call as above for the city of London using the pyowm package

In [83]:
mgr = owm.weather_manager()
observation = mgr.weather_at_place('London,GB')  # the observation object is a box containing a weather object
weather = observation.weather
print(weather.status)           # short version of status (eg. 'Rain')
print(weather.detailed_status)  # detailed version of status (eg. 'light rain')

wind_dict_in_meters_per_sec = observation.weather.wind() 

Rain
light rain


In [84]:
temp_kelvin = weather.temperature()
# temp_dict_fahrenheit = weather.temperature('fahrenheit')  # a dict in Fahrenheit units
# temp_dict_celsius = weather.temperature('celsius')   # a dict in celisius units
temp_kelvin

{'temp': 291.43,
 'temp_max': 292.13,
 'temp_min': 290.71,
 'feels_like': 291.69,
 'temp_kf': None}

In [85]:
wind_dict_in_meters_per_sec = weather.wind() 
wind_dict_in_meters_per_sec

# wind_dict_in_miles_per_h = mgr.weather_at_place('Tokyo,JP').wind(unit='miles_hour')
# wind_dict_in_knots = mgr.weather_at_place('Tokyo,JP').wind(unit='knots')
# wind_dict_in_beaufort = mgr.weather_at_place('Tokyo,JP').wind(unit='beaufort') 

{'speed': 6.17, 'deg': 200}

In [88]:
humidity = weather.humidity
humidity

91

In [89]:
print(f"Temperature: {temp_kelvin['temp']}°C, Humidity: {humidity}%, Wind Speed: {wind_dict_in_meters_per_sec['speed']} m/s")

Temperature: 291.43°C, Humidity: 91%, Wind Speed: 6.17 m/s


In [90]:
#################################################
###############################################
#################################################
#################################################

In [91]:
reg = owm.city_id_registry()

TypeError: 'pyowm.commons.cityidregistry' is not a package

In [92]:
reg = owm.city_id_registry()
list_of_locations = reg.locations_for('Tokyo', country='JP', matching='exact')
tokyo = list_of_locations[0]
lat = tokyo.lat   # 35.689499
lon = tokyo.lon   # 139.691711

TypeError: 'pyowm.commons.cityidregistry' is not a package

In [93]:
owm = OWM("a14cdbc91a8575c20f4ddfdfd8e2bc66")
mgr = owm.geocoding_manager()

# geocode London (no country specified) - we'll get many results
list_of_locations = mgr.geocode('London')
a_london = list_of_locations[0]  # taking the first London in the list
a_london.lat
a_london.lon 

-0.1276474

In [94]:
mgr = owm.weather_manager()
one_call = mgr.one_call(lat=a_london.lat, lon=a_london.lon)

one_call.current.humidity

UnauthorizedError: Invalid API Key provided

## use case 3: Combining and chaining calls

Wrappers often offer methods that make multiple calls to batch requests that make sense to batch. And often they offer methods that make sequences of calls that each returns information necessary to make the next call.

For example, to get a weather forecast for a specific city using the raw API you need to first geocode the city to get its latitude and longitude:

In [95]:
city = 'New York'
geocode_url = f'http://api.openweathermap.org/data/2.5/weather?q={city}'
geocode_response = requests.get(geocode_url,params=params).json()

lat = geocode_response['coord']['lat']
lon = geocode_response['coord']['lon']

(lat,lon)

(40.7143, -74.006)

Then, request the weather forecast for that latitude/longitude:

In [96]:
forecast_url = f'http://api.openweathermap.org/data/2.5/forecast?lat={lat}&lon={lon}'
forecast_response = requests.get(forecast_url, params=params).json()

for entry in forecast_response['list']:
    print(f"Time: {entry['dt_txt']}, Temp: {entry['main']['temp']}°C")

Time: 2025-09-03 09:00:00, Temp: 290.64°C
Time: 2025-09-03 12:00:00, Temp: 290.73°C
Time: 2025-09-03 15:00:00, Temp: 294.29°C
Time: 2025-09-03 18:00:00, Temp: 298.25°C
Time: 2025-09-03 21:00:00, Temp: 296.54°C
Time: 2025-09-04 00:00:00, Temp: 293.65°C
Time: 2025-09-04 03:00:00, Temp: 294.28°C
Time: 2025-09-04 06:00:00, Temp: 292.95°C
Time: 2025-09-04 09:00:00, Temp: 292.18°C
Time: 2025-09-04 12:00:00, Temp: 293.01°C
Time: 2025-09-04 15:00:00, Temp: 296.65°C
Time: 2025-09-04 18:00:00, Temp: 298.15°C
Time: 2025-09-04 21:00:00, Temp: 296.54°C
Time: 2025-09-05 00:00:00, Temp: 295.86°C
Time: 2025-09-05 03:00:00, Temp: 295.51°C
Time: 2025-09-05 06:00:00, Temp: 293.1°C
Time: 2025-09-05 09:00:00, Temp: 293.38°C
Time: 2025-09-05 12:00:00, Temp: 293.68°C
Time: 2025-09-05 15:00:00, Temp: 297.37°C
Time: 2025-09-05 18:00:00, Temp: 300.39°C
Time: 2025-09-05 21:00:00, Temp: 301.06°C
Time: 2025-09-06 00:00:00, Temp: 300.05°C
Time: 2025-09-06 03:00:00, Temp: 297.8°C
Time: 2025-09-06 06:00:00, Temp: 297

Two calls: one for geocoding, one for forecasts.
But with pyowm, because this is a common operation, there is a method that handles the geocoding internally and then fetches the weather forecast in one step.

Get the above forecast in a single call using pyowm.

Hint: search for "forecast_at_place" in the code recipies of the documentation

In [74]:
three_h_forecast

<pyowm.weatherapi30.forecast.Forecast - reception_time=2025-09-02 10:05:58+00:00, interval=3h>

In [97]:
from pyowm.utils import timestamps

three_h_forecast = mgr.forecast_at_place('New York', '3h').forecast

for w in three_h_forecast:
    temp_c = w.temperature()['temp']
    print("Time:", w.reference_time('iso'),"Temp:", temp_c)

Time: 2025-09-03 09:00:00+00:00 Temp: 290.56
Time: 2025-09-03 12:00:00+00:00 Temp: 290.67
Time: 2025-09-03 15:00:00+00:00 Temp: 294.26
Time: 2025-09-03 18:00:00+00:00 Temp: 298.25
Time: 2025-09-03 21:00:00+00:00 Temp: 296.54
Time: 2025-09-04 00:00:00+00:00 Temp: 293.65
Time: 2025-09-04 03:00:00+00:00 Temp: 294.28
Time: 2025-09-04 06:00:00+00:00 Temp: 292.95
Time: 2025-09-04 09:00:00+00:00 Temp: 292.18
Time: 2025-09-04 12:00:00+00:00 Temp: 293.01
Time: 2025-09-04 15:00:00+00:00 Temp: 296.65
Time: 2025-09-04 18:00:00+00:00 Temp: 298.15
Time: 2025-09-04 21:00:00+00:00 Temp: 296.54
Time: 2025-09-05 00:00:00+00:00 Temp: 295.86
Time: 2025-09-05 03:00:00+00:00 Temp: 295.51
Time: 2025-09-05 06:00:00+00:00 Temp: 293.1
Time: 2025-09-05 09:00:00+00:00 Temp: 293.38
Time: 2025-09-05 12:00:00+00:00 Temp: 293.68
Time: 2025-09-05 15:00:00+00:00 Temp: 297.37
Time: 2025-09-05 18:00:00+00:00 Temp: 300.39
Time: 2025-09-05 21:00:00+00:00 Temp: 301.06
Time: 2025-09-06 00:00:00+00:00 Temp: 300.05
Time: 2025-

## use case 4: Convenience methods

Wrappers often offer built-in methods to handle common kinds of tasks related to the APIs, reducing the need for manual calculations.

for example converting units (e.g., temperature from Celsius to Fahrenheit) or working with more complex data requires manual conversion when using the raw API.

In [75]:
city = 'London'
url = f'http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}'

response = requests.get(url)
data = response.json()
temperature_celsius = data['main']['temp']
temperature_fahrenheit = (temperature_celsius * 9/5) + 32

print(f"Temperature in Celsius: {temperature_celsius}°C, Fahrenheit: {temperature_fahrenheit}°F")

Temperature in Celsius: 291.37°C, Fahrenheit: 556.466°F


But the pyowm wrapper offers built-in methods to handle these kinds of tasks, reducing the need for manual calculations.
Get the temperature both in Celcius and Farenheit using pyowm. Navigate the code recipes to figure out the inbuilt methods for this.

In [76]:
observation = mgr.weather_at_place("London,GB")
weather = observation.weather

temp_dict_celsius = weather.temperature('celsius')
temp_dict_fahrenheit = weather.temperature('fahrenheit')

print(f"Temperature in Celsius: {temp_dict_celsius['temp']}°C")
print(f"Temperature in Fahrenheit: {temp_dict_fahrenheit['temp']}°F")

Temperature in Celsius: 18.03°C
Temperature in Fahrenheit: 64.45°F


In [78]:
# Lists of old methods, new methods, and descriptions
old_methods = [
    "get_status()",
    "get_detailed_status()",
    "get_reference_time('iso')",
    "get_reference_time()",
    "get_temperature('celsius')",
    "get_humidity()",
    "get_wind()",
    "get_clouds()",
    "get_rain()",
    "get_snow()"
]

new_methods = [
    "status",
    "detailed_status",
    "reference_time('iso')",
    "reference_time()",
    "temperature('celsius')",
    "humidity",
    "wind()",
    "clouds",
    "rain",
    "snow"
]

descriptions = [
    "Short weather description, e.g., 'Clouds'",
    "Detailed weather description, e.g., 'scattered clouds'",
    "Reference time as ISO-formatted string",
    "Reference time as Unix timestamp",
    "Temperature dictionary: {'temp', 'temp_min', 'temp_max'}",
    "Humidity percentage",
    "Wind dictionary: {'speed', 'deg'}",
    "Cloud coverage percentage",
    "Rain volume dictionary",
    "Snow volume dictionary"
]

# Example of creating a DataFrame
import pandas as pd

df = pd.DataFrame({
    "Old Method": old_methods,
    "New Method": new_methods,
    "Description": descriptions
})
pd.set_option('display.max_colwidth', None)

df


,Old Method,New Method,Description
0,get_status(),status,"Short weather description, e.g., 'Clouds'"
1,get_detailed_status(),detailed_status,"Detailed weather description, e.g., 'scattered clouds'"
2,get_reference_time('iso'),reference_time('iso'),Reference time as ISO-formatted string
3,get_reference_time(),reference_time(),Reference time as Unix timestamp
4,get_temperature('celsius'),temperature('celsius'),"Temperature dictionary: {'temp', 'temp_min', 'temp_max'}"
5,get_humidity(),humidity,Humidity percentage
6,get_wind(),wind(),"Wind dictionary: {'speed', 'deg'}"
7,get_clouds(),clouds,Cloud coverage percentage
8,get_rain(),rain,Rain volume dictionary
9,get_snow(),snow,Snow volume dictionary
